In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [ ]:
# Function to extract ordered hourly variables
def get_ordered_hourly_variables(params):
    return params.get("hourly", [])

# Set up API client with caching and retries
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


def get_reading(url, latitude, longtitute, hourly, past_days):
  params = {
    "latitude": latitude,
    "longitude": longtitute,
    "hourly": hourly,
    "past_days": past_days,
    "forecast_days": 0
  }

  response = openmeteo.weather_api(url, params=params)[0]
  hourly = response.Hourly()
  hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )
  }
  for i, var_name in enumerate(get_ordered_hourly_variables(params)):
    hourly_data[var_name] = hourly.Variables(i).ValuesAsNumpy()

  #Modify df
  hourly_df = pd.DataFrame(data=hourly_data)
  hourly_df['Date/Time'] = pd.to_datetime(hourly_df['date'], errors='coerce')
  hourly_df.set_index('Date/Time', inplace=True)
  hourly_df.drop(columns=['date'], inplace = True)

  return hourly_df

######Sample use#############


get_reading("https://api.open-meteo.com/v1/forecast",
            31.990534, 35.907787,
             [
           "temperature_2m", "dew_point_2m", "relative_humidity_2m",
           "rain", "pressure_msl", "wind_speed_10m", "wind_direction_10m"],
            5)
